# Requirements

In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 25.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import polars as pl

In [4]:
import re

# Reading Data

In [14]:
dft = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')

# Analysis

In [15]:
pl.read_parquet_schema('/kaggle/input/aeroclub-recsys-2025/train.parquet')

Schema([('Id', Int64),
        ('bySelf', Boolean),
        ('companyID', Int64),
        ('corporateTariffCode', Int64),
        ('frequentFlyer', String),
        ('nationality', Int64),
        ('isAccess3D', Boolean),
        ('isVip', Boolean),
        ('legs0_arrivalAt', String),
        ('legs0_departureAt', String),
        ('legs0_duration', String),
        ('legs0_segments0_aircraft_code', String),
        ('legs0_segments0_arrivalTo_airport_city_iata', String),
        ('legs0_segments0_arrivalTo_airport_iata', String),
        ('legs0_segments0_baggageAllowance_quantity', Float64),
        ('legs0_segments0_baggageAllowance_weightMeasurementType', Float64),
        ('legs0_segments0_cabinClass', Float64),
        ('legs0_segments0_departureFrom_airport_iata', String),
        ('legs0_segments0_duration', String),
        ('legs0_segments0_flightNumber', String),
        ('legs0_segments0_marketingCarrier_code', String),
        ('legs0_segments0_operatingCarrier_code', Str

In [16]:
dft

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf696

# Cleaning Data & Feature Engineering

In [17]:
dft= dft.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code')

In [18]:
dft = dft.drop('legs1_segments0_duration')

In [19]:
dft = dft.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [20]:
dft['TotalPrice']

TotalPrice
f64
17254.0
53365.0
55935.0
84120.0
88310.0
…
36290.0
33220.0
29920.0


In [21]:
dft = dft.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [22]:
dft = dft.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [23]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15T15:40:00""","""2024-06-15T16:20:00""","""02:40:00""","""2024-07-09T09:45:00""","""2024-07-09T14:20:00""","""02:35:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
"""2024-06-15T09:25:00""","""2024-06-15T14:50:00""","""07:25:00""","""2024-07-09T22:05:00""","""2024-07-10T08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05T20:50:00""","""2024-11-06T09:45:00""","""10:55:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T21:10:00""","""2024-11-10T08:35:00""","""13:25:00"""
"""2024-11-05T00:20:00""","""2024-11-05T20:00:00""","""17:40:00""","""2024-11-09T19:25:00""","""2024-11-10T08:10:00""","""14:45:00"""


In [24]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [25]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-06-15 15:40:00""","""2024-06-15 16:20:00""","""02:40:00""","""2024-07-09 09:45:00""","""2024-07-09 14:20:00""","""02:35:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
"""2024-06-15 09:25:00""","""2024-06-15 14:50:00""","""07:25:00""","""2024-07-09 22:05:00""","""2024-07-10 08:30:00""","""08:25:00"""
…,…,…,…,…,…
"""2024-11-05 20:50:00""","""2024-11-06 09:45:00""","""10:55:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 21:10:00""","""2024-11-10 08:35:00""","""13:25:00"""
"""2024-11-05 00:20:00""","""2024-11-05 20:00:00""","""17:40:00""","""2024-11-09 19:25:00""","""2024-11-10 08:10:00""","""14:45:00"""


In [26]:
dft= dft.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [27]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt']

legs0_departureAt,legs0_arrivalAt,legs1_departureAt,legs1_arrivalAt
datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2024-06-15 15:40:00,2024-06-15 16:20:00,2024-07-09 09:45:00,2024-07-09 14:20:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
2024-06-15 09:25:00,2024-06-15 14:50:00,2024-07-09 22:05:00,2024-07-10 08:30:00
…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,2024-11-09 19:25:00,2024-11-10 08:10:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 21:10:00,2024-11-10 08:35:00
2024-11-05 00:20:00,2024-11-05 20:00:00,2024-11-09 19:25:00,2024-11-10 08:10:00


In [28]:
dft['legs0_duration',
'legs1_duration']

legs0_duration,legs1_duration
str,str
"""02:40:00""","""02:35:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
"""07:25:00""","""08:25:00"""
…,…
"""10:55:00""","""14:45:00"""
"""17:40:00""","""13:25:00"""
"""17:40:00""","""14:45:00"""


In [29]:
dft = dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [30]:
dft=dft.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [31]:
dft['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-06-15 15:40:00,2024-06-15 16:20:00,02:40:00,2024-07-09 09:45:00,2024-07-09 14:20:00,02:35:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
2024-06-15 09:25:00,2024-06-15 14:50:00,07:25:00,2024-07-09 22:05:00,2024-07-10 08:30:00,08:25:00
…,…,…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,10:55:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 21:10:00,2024-11-10 08:35:00,13:25:00
2024-11-05 00:20:00,2024-11-05 20:00:00,17:40:00,2024-11-09 19:25:00,2024-11-10 08:10:00,14:45:00


In [5]:
dftt = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')

In [6]:
dftt= dftt.drop('bySelf','sex','legs0_segments0_cabinClass','nationality','legs0_segments0_marketingCarrier_code','legs0_segments2_marketingCarrier_code','legs0_segments3_marketingCarrier_code','legs1_segments0_marketingCarrier_code','legs1_segments2_marketingCarrier_code','legs1_segments3_marketingCarrier_code',)

In [7]:
dftt = dftt.with_columns(
    TotalPrice = pl.col("totalPrice") + pl.col("taxes"))

In [8]:
dftt = dftt.drop('legs1_segments0_duration')

In [9]:
dftt = dftt.drop('legs0_segments0_duration','legs0_segments1_duration','legs0_segments2_duration','legs0_segments3_duration')

In [10]:
dftt = dftt.drop('legs1_segments1_duration','legs1_segments2_duration','legs1_segments3_duration')

In [11]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
str,str,str,str,str,str
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T06:50:00""","""2024-12-19T11:20:00""","""02:30:00""","""2024-12-21T21:10:00""","""2024-12-21T21:55:00""","""02:45:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
"""2024-12-19T08:25:00""","""2024-12-19T12:45:00""","""02:20:00""","""2024-12-21T12:00:00""","""2024-12-21T12:35:00""","""02:35:00"""
…,…,…,…,…,…
"""2025-01-08T09:05:00""","""2025-01-08T12:50:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null
"""2025-01-08T21:25:00""","""2025-01-09T01:10:00""","""02:45:00""",null,null,null


In [12]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration').str.replace(r"T", "  "))

In [13]:
dftt= dftt.with_columns(
    pl.col('legs0_departureAt',
'legs0_arrivalAt',
'legs1_departureAt',
'legs1_arrivalAt').str.to_datetime("%Y-%m-%d %H:%M:%S"))

In [14]:
dftt = dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.replace_all(r"\.\d{2}", "")
)

In [15]:
dftt=dftt.with_columns(
    pl.col('legs0_duration',
'legs1_duration').str.to_time("%H:%M:%S"))

In [16]:
dftt['legs0_departureAt',
'legs0_arrivalAt',
'legs0_duration',
'legs1_departureAt',
'legs1_arrivalAt',
'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],time,datetime[μs],datetime[μs],time
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 06:50:00,2024-12-19 11:20:00,02:30:00,2024-12-21 21:10:00,2024-12-21 21:55:00,02:45:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
2024-12-19 08:25:00,2024-12-19 12:45:00,02:20:00,2024-12-21 12:00:00,2024-12-21 12:35:00,02:35:00
…,…,…,…,…,…
2025-01-08 09:05:00,2025-01-08 12:50:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null
2025-01-08 21:25:00,2025-01-09 01:10:00,02:45:00,null,null,null


In [17]:
def preprocess(df: pl.DataFrame) -> pl.DataFrame:
    datetime_cols = ["legs0_departureAt", "legs0_arrivalAt",
                     "legs1_departureAt", "legs1_arrivalAt"]

    for col in datetime_cols:
        # Ensure datetime type
        df = df.with_columns(
            pl.col(col).cast(pl.Datetime(time_unit="us"))
        )

        # Basic datetime features
        df = df.with_columns([
            pl.col(col).dt.hour().alias(f"{col}_hour"),
            pl.col(col).dt.weekday().alias(f"{col}_dow"),
            pl.col(col).dt.month().alias(f"{col}_month"),
            (pl.col(col).dt.weekday() >= 5).cast(pl.Int8).alias(f"{col}_is_weekend"),
        ])

        # Cyclical hour encoding
        df = df.with_columns([
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).sin().alias(f"{col}_hour_sin"),
            (pl.col(f"{col}_hour") * (2 * np.pi) / 24).cos().alias(f"{col}_hour_cos"),
        ])

    # Duration to minutes
    duration_cols = ["legs0_duration", "legs1_duration"]
    for col in duration_cols:
    # If already a Duration, just convert to minutes
        if df[col].dtype == pl.Duration(time_unit="us"):
            df = df.with_columns(
                (pl.col(col).dt.seconds() / 60).alias(col)
            )
        # If it's a Time (HH:MM:SS)
        elif df[col].dtype == pl.Time:
            df = df.with_columns(
                (pl.col(col).dt.hour() * 60 +
                 pl.col(col).dt.minute() +
                 pl.col(col).dt.second() / 60).alias(col)
            )
        # If it's a string like "02:40:00"
        else:
            df = df.with_columns(
                (pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.hour() * 60 +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.minute() +
                 pl.col(col).str.strptime(pl.Time, "%H:%M:%S", strict=False).dt.second() / 60
                ).alias(col)
            )
    # Layover
    if all(c in df.columns for c in ["legs1_departureAt", "legs0_arrivalAt"]):
    
        df = df.with_columns(
            (
                pl.col("legs1_departureAt").cast(pl.Datetime("us")) -
                pl.col("legs0_arrivalAt").cast(pl.Datetime("us"))
            ).dt.total_seconds() / 60
            
        )
    return df

In [45]:
print("Preprocessing...")
dft = preprocess(dft)
dftt = preprocess(dftt)

Preprocessing...


In [18]:
dftt = preprocess(dftt)

# Feature Selection

In [46]:
x = dft['Id','TotalPrice','searchRoute','corporateTariffCode','profileId','companyID',
'legs0_duration' ,
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs0_segments2_seatsAvailable','legs0_segments3_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable','legs1_segments2_seatsAvailable','legs1_segments3_seatsAvailable',
'miniRules0_monetaryAmount' ,
'miniRules0_percentage',
'miniRules0_statusInfos' ,
'miniRules1_monetaryAmount' ,
'miniRules1_percentage' ,
'miniRules1_statusInfos' ,
'pricingInfo_isAccessTP' ,
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs0_segments2_baggageAllowance_quantity','legs0_segments3_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity','legs1_segments2_baggageAllowance_quantity','legs1_segments3_baggageAllowance_quantity'
,'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata','legs0_segments2_arrivalTo_airport_iata','legs0_segments3_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata','legs1_segments2_arrivalTo_airport_iata','legs1_segments3_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata','legs0_segments2_arrivalTo_airport_city_iata','legs0_segments3_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata','legs1_segments2_arrivalTo_airport_city_iata','legs1_segments3_arrivalTo_airport_city_iata']

In [47]:
string_columns = [col_name for col_name, dtype in zip(x.columns, x.dtypes) if dtype == pl.Utf8]
print(string_columns)

['searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments2_arrivalTo_airport_iata', 'legs0_segments3_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments2_arrivalTo_airport_iata', 'legs1_segments3_arrivalTo_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments2_arrivalTo_airport_city_iata', 'legs0_segments3_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'legs1_segments2_arrivalTo_airport_city_iata', 'legs1_segments3_arrivalTo_airport_city_iata']


In [48]:
CATEGORICAL_COLS = ['searchRoute', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments2_arrivalTo_airport_iata', 'legs0_segments3_arrivalTo_airport_iata', 'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments2_arrivalTo_airport_iata', 'legs1_segments3_arrivalTo_airport_iata', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments2_arrivalTo_airport_city_iata', 'legs0_segments3_arrivalTo_airport_city_iata', 'legs1_segments0_arrivalTo_airport_city_iata', 'legs1_segments1_arrivalTo_airport_city_iata', 'legs1_segments2_arrivalTo_airport_city_iata', 'legs1_segments3_arrivalTo_airport_city_iata']


In [19]:
xt = dftt['Id','TotalPrice','searchRoute','corporateTariffCode','profileId'
,'companyID','legs0_duration',
'legs1_duration','legs0_segments0_seatsAvailable','legs0_segments1_seatsAvailable','legs0_segments2_seatsAvailable','legs0_segments3_seatsAvailable','legs1_segments0_seatsAvailable','legs1_segments1_seatsAvailable','legs1_segments2_seatsAvailable','legs1_segments3_seatsAvailable',
'miniRules0_monetaryAmount','miniRules0_percentage','miniRules0_statusInfos',
'miniRules1_monetaryAmount','miniRules1_percentage','miniRules1_statusInfos',
'pricingInfo_isAccessTP',
'pricingInfo_passengerCount','legs0_segments0_baggageAllowance_quantity','legs0_segments1_baggageAllowance_quantity','legs0_segments2_baggageAllowance_quantity','legs0_segments3_baggageAllowance_quantity','legs1_segments0_baggageAllowance_quantity','legs1_segments1_baggageAllowance_quantity','legs1_segments2_baggageAllowance_quantity','legs1_segments3_baggageAllowance_quantity'
,'legs0_segments0_arrivalTo_airport_iata','legs0_segments1_arrivalTo_airport_iata','legs0_segments2_arrivalTo_airport_iata','legs0_segments3_arrivalTo_airport_iata',
'legs1_segments0_arrivalTo_airport_iata','legs1_segments1_arrivalTo_airport_iata','legs1_segments2_arrivalTo_airport_iata','legs1_segments3_arrivalTo_airport_iata',
'legs0_segments0_arrivalTo_airport_city_iata','legs0_segments1_arrivalTo_airport_city_iata','legs0_segments2_arrivalTo_airport_city_iata','legs0_segments3_arrivalTo_airport_city_iata',
'legs1_segments0_arrivalTo_airport_city_iata','legs1_segments1_arrivalTo_airport_city_iata','legs1_segments2_arrivalTo_airport_city_iata','legs1_segments3_arrivalTo_airport_city_iata']

In [50]:
TARGET_COL = "selected"
GROUP_COL = "ranker_id"
MODEL_OUT = "flight_ranker.cbm"
PREDICTIONS_OUT = "ranked_predictions.csv"

In [51]:
exclude_cols = [TARGET_COL, GROUP_COL]
feature_cols = [c for c in x.columns if c not in exclude_cols]

# CatBoost categorical column indices
cat_feature_indices = [feature_cols.index(c) for c in CATEGORICAL_COLS if c in feature_cols]

In [52]:
feature_cols

['Id',
 'TotalPrice',
 'searchRoute',
 'corporateTariffCode',
 'profileId',
 'companyID',
 'legs0_duration',
 'legs1_duration',
 'legs0_segments0_seatsAvailable',
 'legs0_segments1_seatsAvailable',
 'legs0_segments2_seatsAvailable',
 'legs0_segments3_seatsAvailable',
 'legs1_segments0_seatsAvailable',
 'legs1_segments1_seatsAvailable',
 'legs1_segments2_seatsAvailable',
 'legs1_segments3_seatsAvailable',
 'miniRules0_monetaryAmount',
 'miniRules0_percentage',
 'miniRules0_statusInfos',
 'miniRules1_monetaryAmount',
 'miniRules1_percentage',
 'miniRules1_statusInfos',
 'pricingInfo_isAccessTP',
 'pricingInfo_passengerCount',
 'legs0_segments0_baggageAllowance_quantity',
 'legs0_segments1_baggageAllowance_quantity',
 'legs0_segments2_baggageAllowance_quantity',
 'legs0_segments3_baggageAllowance_quantity',
 'legs1_segments0_baggageAllowance_quantity',
 'legs1_segments1_baggageAllowance_quantity',
 'legs1_segments2_baggageAllowance_quantity',
 'legs1_segments3_baggageAllowance_quantity',


In [53]:
cat_feature_indices

[2, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

In [54]:
dft['legs0_departureAt',
 'legs0_arrivalAt',
 'legs0_duration',
 'legs1_departureAt',
 'legs1_arrivalAt',
 'legs1_duration']

legs0_departureAt,legs0_arrivalAt,legs0_duration,legs1_departureAt,legs1_arrivalAt,legs1_duration
datetime[μs],datetime[μs],f64,f64,datetime[μs],f64
2024-06-15 15:40:00,2024-06-15 16:20:00,-96.0,34165.0,2024-07-09 14:20:00,-101.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,34995.0,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,34995.0,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,34995.0,2024-07-10 08:30:00,-7.0
2024-06-15 09:25:00,2024-06-15 14:50:00,-67.0,34995.0,2024-07-10 08:30:00,-7.0
…,…,…,…,…,…
2024-11-05 20:50:00,2024-11-06 09:45:00,-113.0,4900.0,2024-11-10 08:10:00,117.0
2024-11-05 00:20:00,2024-11-05 20:00:00,36.0,5830.0,2024-11-10 08:35:00,37.0
2024-11-05 00:20:00,2024-11-05 20:00:00,36.0,5725.0,2024-11-10 08:10:00,117.0


In [22]:
test = dftt.write_parquet("/kaggle/working/test.parquet")

# Trainig Model

In [55]:
! pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 4.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 80.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 32.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [56]:
import numpy as np
from catboost import CatBoostRanker, Pool

In [57]:
# 1. Check for nulls in critical columns
print("NaNs in target:", dft[TARGET_COL].is_null().sum())
print("NaNs in group_id:", dft[GROUP_COL].is_null().sum())

# 2. Fill missing group_id with a placeholder
dft = dft.with_columns(
    pl.col(GROUP_COL).fill_null(-1)  # or some valid integer ID
)

# 3. Fill or drop missing target
dft = dft.filter(pl.col(TARGET_COL).is_not_null())  # safest for classification

# 4. CatBoost can handle NaNs in numeric features, but ensure proper type casting for cat_features
for cat_col in cat_feature_indices:
    dft = dft.with_columns(pl.col(feature_cols[cat_col]).cast(pl.Utf8))


NaNs in target: 0
NaNs in group_id: 0


In [58]:
for col in feature_cols:
    if dft[col].dtype in [pl.Int64, pl.Int32, pl.UInt32, pl.UInt64, pl.Float64, pl.Float32]:
        dft = dft.with_columns(pl.col(col).cast(pl.Float64))
    else:
        # keep categorical columns as string
        dft = dft.with_columns(pl.col(col).cast(pl.Utf8))

# Fill nulls in numeric cols with np.nan explicitly
dft = dft.fill_null(np.nan)

# Make sure group_id and label have no NaNs
dft = dft.filter(
    (pl.col(GROUP_COL).is_not_null()) &
    (pl.col(TARGET_COL).is_not_null())
)

In [59]:
for col in feature_cols:
    if dft[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                          pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                          pl.Float32, pl.Float64]:
        dft = dft.with_columns(pl.col(col).cast(pl.Float64))

# Replace nulls in numeric with np.nan
dft = dft.fill_null(np.nan)

# Drop rows with missing label or group_id
dft = dft.filter(
    pl.col(TARGET_COL).is_not_null() & pl.col(GROUP_COL).is_not_null()
)

In [60]:
import numpy as np
import polars as pl
from catboost import Pool

# 1. Ensure numeric columns are floats → np.nan
for col in feature_cols:
    if dft[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                          pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                          pl.Float32, pl.Float64]:
        dft = dft.with_columns(pl.col(col).cast(pl.Float64))

# 2. Handle categorical columns
for idx in cat_feature_indices:
    col = feature_cols[idx]
    dft = dft.with_columns(pl.col(col).cast(pl.Utf8))
    dft = dft.with_columns(pl.col(col).fill_null("__MISSING__"))

# 3. Fill remaining nulls in numeric with np.nan
dft = dft.fill_null(np.nan)

# 4. Drop rows where label or group_id is null
dft = dft.filter(
    pl.col(TARGET_COL).is_not_null() & pl.col(GROUP_COL).is_not_null()
)

# 5. Create Pool
train_pool = Pool(
    data=dft.select(feature_cols).to_numpy(),
    label=dft[TARGET_COL].to_numpy(),
    group_id=dft[GROUP_COL].to_numpy(),
    cat_features=cat_feature_indices
)


In [61]:
# 1. Check for nulls in critical columns
print("NaNs in group_id:", dftt[GROUP_COL].is_null().sum())

# 2. Fill missing group_id with a placeholder
dftt = dftt.with_columns(
    pl.col(GROUP_COL).fill_null(-1)  # or some valid integer ID
)


# 4. CatBoost can handle NaNs in numeric features, but ensure proper type casting for cat_features
for cat_col in cat_feature_indices:
    dftt = dftt.with_columns(pl.col(feature_cols[cat_col]).cast(pl.Utf8))


NaNs in group_id: 0


In [62]:
for col in feature_cols:
    if dftt[col].dtype in [pl.Int64, pl.Int32, pl.UInt32, pl.UInt64, pl.Float64, pl.Float32]:
        dftt = dftt.with_columns(pl.col(col).cast(pl.Float64))
    else:
        # keep categorical columns as string
        dftt = dftt.with_columns(pl.col(col).cast(pl.Utf8))

# Fill nulls in numeric cols with np.nan explicitly
dftt = dftt.fill_null(np.nan)

# Make sure group_id and label have no NaNs
dftt = dftt.filter(
    (pl.col(GROUP_COL).is_not_null()) )
   


In [63]:
for col in feature_cols:
    if dftt[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                          pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                          pl.Float32, pl.Float64]:
        dftt = dftt.with_columns(pl.col(col).cast(pl.Float64))

# Replace nulls in numeric with np.nan
dftt = dftt.fill_null(np.nan)

# Drop rows with missing label or group_id
dftt = dftt.filter(
    pl.col(GROUP_COL).is_not_null()
)

In [64]:
# 1. Ensure numeric columns are floats → np.nan
for col in feature_cols:
    if dftt[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                          pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                          pl.Float32, pl.Float64]:
        dftt = dftt.with_columns(pl.col(col).cast(pl.Float64))

# 2. Handle categorical columns
for idx in cat_feature_indices:
    col = feature_cols[idx]
    dftt = dftt.with_columns(pl.col(col).cast(pl.Utf8))
    dftt = dftt.with_columns(pl.col(col).fill_null("__MISSING__"))

# 3. Fill remaining nulls in numeric with np.nan
dftt = dftt.fill_null(np.nan)

# 4. Drop rows where label or group_id is null
dftt = dftt.filter(
     pl.col(GROUP_COL).is_not_null()
)
test_pool = Pool(
    data=dftt.select(feature_cols).to_numpy(),
    group_id=dftt[GROUP_COL].to_numpy(),
    cat_features=cat_feature_indices
)


In [ ]:
print("Training CatBoostRanker...")
model = CatBoostRanker(
    iterations=300,
    learning_rate=0.05,
    depth=8,
    loss_function="YetiRank",
    eval_metric="NDCG",
    random_seed=42,
    verbose=100
)
model.fit(train_pool)
model.save_model('/kaggle/working/MODEL_OUT.pkl')

Training CatBoostRanker...
Groupwise loss function. OneHotMaxSize set to 10
0:	total: 19.5s	remaining: 1h 37m 9s


In [ ]:
# Predict & rank
print("Predicting...")
scores = model.predict(test_pool)
test_df = dftt.with_columns(pl.Series("score", scores))



In [ ]:
print("Ranking per group...")
test_df = test_df.with_columns(
    pl.col("score").rank("dense", descending=True).over(GROUP_COL).alias("rank")
)

# Save
test_df.sort(['Id',GROUP_COL, "rank"]).write_csv('/kaggle/working/PREDICTIONS_OUT.csv')
print(f"Predictions saved to {PREDICTIONS_OUT}")

In [ ]:
import  pandas as pd
z = pd.read_csv('/kaggle/working/PREDICTIONS_OUT.csv')
z.shape
z.head(20)